<a href="https://colab.research.google.com/github/j-ranasinghe/nlp_cw/blob/vishmi/Solution_GroupH.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## CM4603 – Coursework 1 (Group)
#### November 2023

### Group: H

Name 1: Anuttara Rajasinghe |
IIT ID: 20210216 |
RGU ID: 2117946

Name 2: < insert your name > |
IIT ID: < insert your IIT ID > |
RGU ID: < insert your RGU ID >

Name 3: Vishmi Herath |
IIT ID: 20210503 |
RGU ID: 2117528

### Answer to Task 1

    Describe the dataset, any insights you gained about the data and the effectiveness of the different (subword) tokenization schemes tried out.

In [ ]:
# Insert code for Task 1

Explain the challenges encountered and the observations based on the output above

### Answer to Task 2

Describe the classification dataset, its dimensions, preprocessing steps, numbers of classes taken for classification, the which feature extraction techniques were performed.

In [ ]:
# Insert code for Task 2

Describe the final dataset shapes of the featuresets extracted.

### Answer to Task 3

Justify the 3 non-deep learning algorithms you are using for classification.

In [ ]:
# Insert code for Task 3

Compare and interpret the results of the different algorithms and feature extraction methods on the text classification task.

### Answer to Task 4

Describe the contextual embedding used and it's effect on the performance of the algorithms selected in Task 3.

In [ ]:
# Insert code for Task 4

Compare the deep learning algorithm results with that of the 3 non-deep learning algorithms and interpret the overal results